<a href="https://colab.research.google.com/github/Yeyen09/youtube-comment-sentimental-analysis/blob/main/Senti1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Mengimpor pustaka yang diperlukan
import csv #Untuk membaca dan menulis file CSV
import re #untuk regular expressions
import pandas as pd #Untuk manipulasi dan analisis data
import nltk # Natural Language Toolkit untuk pemrosesan teks
nltk.download('vader_lexicon') #Download VADER lexicon untuk analisis sentimen
from nltk.sentiment.vader import SentimentIntensityAnalyzer #Sentiment analysis tool
import plotly.express as px #Untuk membuat plot interaktif
import plotly.graph_objects as go #Untuk membuat plot terperinci
from colorama import Fore, Style #Untuk output berwarna di terminal
from typing import Dict #Untuk mengetik kamus petunjuk
import streamlit as st #Untuk membuat aplikasi web

In [8]:
#Berfungsi untuk mengekstrak ID video dari tautan YouTube
def extract_video_id(youtube_link):
  #Ekspresi reguler untuk mencocokkan ID video YouTube
    video_id_regex = r"^(?:https?:\/\/)?(?:www\.)?(?:youtube\.com\/watch\?v=|youtu.be\/)([a-zA-Z0-9_-]{11})"
    match = re.search(video_id_regex, youtube_link) #Cari pola dalam tautan tersebut
    if match:
        video_id = match.group(1) #Mengekstrak ID video jika polanya cocok
        return video_id
    else:
        return None #mengembalikan none jika tidak ada kecocokan yang ditemukan

In [ ]:
#Berfungsi untuk menganalisis sentimen dari komentar dalam file CSV
def analyze_sentiment(csv_file):
    #Inisialisasi analisa sentimen
    sid = SentimentIntensityAnalyzer()

    #Baca di komentar YouTube dari file CSV
    comments = []
    with open(csv_file, 'r', encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            comments.append(row['Comment']) #Menambahkan setiap komentar ke dalam daftar

    #Menghitung jumlah komentar netral, positif, dan negatif
    num_neutral = 0
    num_positive = 0
    num_negative = 0
    for comment in comments:
        sentiment_scores = sid.polarity_scores(comment) #Dapatkan nilai sentimen untuk komentar
        if sentiment_scores['compound'] == 0.0:
            num_neutral += 1 #Increment neutral count
        elif sentiment_scores['compound'] > 0.0:
            num_positive += 1 #Increment positive count
        else:
            num_negative += 1 #Increment negative count

    #Mengembalikan hasil sebagai kamus
    results = {'num_neutral': num_neutral, 'num_positive': num_positive, 'num_negative': num_negative}
    return results

In [ ]:
def bar_chart(csv_file: str) -> None:
    #Memanggil fungsi analyze_sentiment untuk mendapatkan hasil analisis sentimen
    results: Dict[str, int] = analyze_sentiment(csv_file)

    #Mendapatkan jumlah komentar untuk setiap kategori sentimen
    num_neutral = results['num_neutral']
    num_positive = results['num_positive']
    num_negative = results['num_negative']

    #Membuat DataFrame Pandas dengan hasil analisis sentimen
    df = pd.DataFrame({
        'Sentiment': ['Positive', 'Negative', 'Neutral'],
        'Number of Comments': [num_positive, num_negative, num_neutral]
    })

    #Membuat diagram batang menggunakan Plotly Express
    fig = px.bar(df, x='Sentiment', #Sumbu x adalah kategori sentimen
                 y='Number of Comments', #Sumbu y adalah jumlah komentar
                 color='Sentiment', #Pewarnaan berdasarkan kategori sentimen
                 color_discrete_sequence=['#87CEFA', '#FFA07A', '#D3D3D3'],#Warna khusus untuk setiap kategori
                 title='Sentiment Analysis Results') #Judul diagram
    fig.update_layout(title_font=dict(size=20)) #Mengatur ukuran font judul


    #Menampilkan diagram di Streamlit
    st.plotly_chart(fig, use_container_width=True)

In [ ]:
def plot_sentiment(csv_file: str) -> None:
    #Memanggil fungsi analyze_sentiment untuk mendapatkan hasil analisis sentimen
    results: Dict[str, int] = analyze_sentiment(csv_file)

    #Mendapatkan jumlah komentar untuk setiap kategori sentimen
    num_neutral = results['num_neutral']
    num_positive = results['num_positive']
    num_negative = results['num_negative']

    #Mempersiapkan data untuk diagram pie (pie chart)
    labels = ['Neutral', 'Positive', 'Negative'] #Label untuk setiap kategori sentimen
    values = [num_neutral, num_positive, num_negative] #Nilai jumlah komentar untuk setiap kategori
    colors = ['yellow', 'green', 'red'] #Warna yang digunakan untuk setiap kategori
    #Membuat diagram pie menggunakan Plotly
    fig = go.Figure(data=[go.Pie(labels=labels, values=values, textinfo='label+percent',
                                 marker=dict(colors=colors))]) #Membuat diagram pie dengan label, nilai, dan warna
    #Mengatur tata letak diagram
    fig.update_layout(title={'text': 'Sentiment Analysis Results', #Judul diagram
                             'font': {'size': 20, 'family': 'Arial', 'color': 'grey'}, #Pengaturan font judul
                              'x': 0.5, 'y': 0.9}, #Posisi judul (di tengah atas)
                      font=dict(size=14)) #Pengaturan ukuran font umum
    st.plotly_chart(fig) #Menampilkan diagram di Streamlit

In [ ]:
#Berfungsi untuk membuat scatter plot dari file CSV
def create_scatterplot(csv_file: str, x_column: str, y_column: str) -> None:
    #Memuat data dari CSV
    data = pd.read_csv(csv_file)

    #Membuat scatter plot menggunakan Plotly
    fig = px.scatter(data, x=x_column, y=y_column, color='Category')

    #Menyesuaikan tata letak
    fig.update_layout(
        title='Scatter Plot',
        xaxis_title=x_column,
        yaxis_title=y_column,
        font=dict(size=18)
    )

    #Menampilkan plot dalam Streamlit
    st.plotly_chart(fig, use_container_width=True)

In [ ]:
def print_sentiment(csv_file: str) -> None:
    #Memanggil fungsi analyze_sentiment untuk mendapatkan hasil analisis sentimen
    results: Dict[str, int] = analyze_sentiment(csv_file)

    #Mendapatkan jumlah komentar untuk setiap kategori sentimen
    num_neutral = results['num_neutral']
    num_positive = results['num_positive']
    num_negative = results['num_negative']


    #Menentukan sentimen keseluruhan berdasarkan perbandingan jumlah komentar positif dan negatif
    if num_positive > num_negative:
        overall_sentiment = 'POSITIVE' #Jika komentar positif lebih banyak, maka sentimen keseluruhan adalah positif
        color = Fore.GREEN #Warna hijau untuk sentimen positif
    elif num_negative > num_positive:
        overall_sentiment = 'NEGATIVE' #Jika komentar negatif lebih banyak, maka sentimen keseluruhan adalah negatif
        color = Fore.RED #Warna merah untuk sentimen negatif
    else:
        overall_sentiment = 'NEUTRAL' #Jika jumlah komentar positif dan negatif seimbang, maka sentimen keseluruhan adalah netral
        color = Fore.YELLOW #Warna kuning untuk sentimen netra

    #Mencetak sentimen keseluruhan dengan warna yang sesuai
    print('\n'+ Style.BRIGHT+ color + overall_sentiment.upper().center(50, ' ') + Style.RESET_ALL)
    #Mencetak sentimen dalam huruf besar (uppercase) dan menempatkannya di tengah dengan padding
    #Menggunakan warna yang telah ditentukan dan mengatur gaya cetak menjadi terang (bright)
    #Mengembalikan gaya cetak ke pengaturan normal setelah mencetak